In [1]:
import numpy as np
import pandas as pd
import os, sys
import json
import pprint

In [2]:
dataset_name = 'spotify_genre'
inp_fname = 'genres_v2.csv'

In [3]:
input_dir = './raw'
output_dir = './processed'

outp_fname = os.path.join(output_dir, f'{dataset_name}.csv')
outp_train_fname = os.path.join(output_dir, f'{dataset_name}_train.csv')
outp_test_fname = os.path.join(output_dir, f'{dataset_name}_test.csv')
outp_test_key_fname = os.path.join(output_dir, f'{dataset_name}_test_key.csv')
outp_infer_instances = os.path.join(output_dir, f'{dataset_name}_infer_req.json')

# Read Data

In [4]:
data = pd.read_csv(os.path.join(input_dir, inp_fname))
data.head()

C:\Users\abbus\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,genre,song_name,Unnamed: 0,title
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,...,2Vc6NJ9PW9gD9q343XFRKx,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,https://api.spotify.com/v1/tracks/2Vc6NJ9PW9gD...,https://api.spotify.com/v1/audio-analysis/2Vc6...,124539,4,Dark Trap,Mercury: Retrograde,NaN,NaN
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,...,7pgJBLVz5VmnL7uGHmRj6p,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,https://api.spotify.com/v1/tracks/7pgJBLVz5Vmn...,https://api.spotify.com/v1/audio-analysis/7pgJ...,224427,4,Dark Trap,Pathology,NaN,NaN
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,...,0vSWgAlfpye0WCGeNmuNhy,spotify:track:0vSWgAlfpye0WCGeNmuNhy,https://api.spotify.com/v1/tracks/0vSWgAlfpye0...,https://api.spotify.com/v1/audio-analysis/0vSW...,98821,4,Dark Trap,Symbiote,NaN,NaN
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,...,0VSXnJqQkwuH2ei1nOQ1nu,spotify:track:0VSXnJqQkwuH2ei1nOQ1nu,https://api.spotify.com/v1/tracks/0VSXnJqQkwuH...,https://api.spotify.com/v1/audio-analysis/0VSX...,123661,3,Dark Trap,ProductOfDrugs (Prod. The Virus and Antidote),NaN,NaN
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,...,4jCeguq9rMTlbMmPHuO7S3,spotify:track:4jCeguq9rMTlbMmPHuO7S3,https://api.spotify.com/v1/tracks/4jCeguq9rMTl...,https://api.spotify.com/v1/audio-analysis/4jCe...,123298,4,Dark Trap,Venom,NaN,NaN


In [5]:
data.shape

(42305, 22)

In [6]:
id_col = "id"
target_col = "genre"

# Insert Id Column

In [7]:
# insert Id column 
if id_col not in data.columns:
    N = data.shape[0]
    data.insert(0, id_col, np.arange(N))
    print(data.head())
data[id_col] = data[id_col].astype(str)

# Prepare Data

In [8]:
data.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'genre', 'song_name', 'Unnamed: 0', 'title'],
      dtype='object')

In [9]:
# drop symbolic variables 

cols_to_drop = ['type', 'uri', 'track_href', 'analysis_url', 'song_name', 'Unnamed: 0', 'title']
data.drop(labels=cols_to_drop, axis=1, inplace=True)

data.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,genre
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,156.985,2Vc6NJ9PW9gD9q343XFRKx,124539,4,Dark Trap
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,115.080,7pgJBLVz5VmnL7uGHmRj6p,224427,4,Dark Trap
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,218.050,0vSWgAlfpye0WCGeNmuNhy,98821,4,Dark Trap
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,186.948,0VSXnJqQkwuH2ei1nOQ1nu,123661,3,Dark Trap
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,147.988,4jCeguq9rMTlbMmPHuO7S3,123298,4,Dark Trap


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42305 entries, 0 to 42304
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      42305 non-null  float64
 1   energy            42305 non-null  float64
 2   key               42305 non-null  int64  
 3   loudness          42305 non-null  float64
 4   mode              42305 non-null  int64  
 5   speechiness       42305 non-null  float64
 6   acousticness      42305 non-null  float64
 7   instrumentalness  42305 non-null  float64
 8   liveness          42305 non-null  float64
 9   valence           42305 non-null  float64
 10  tempo             42305 non-null  float64
 11  id                42305 non-null  object 
 12  duration_ms       42305 non-null  int64  
 13  time_signature    42305 non-null  int64  
 14  genre             42305 non-null  object 
dtypes: float64(9), int64(4), object(2)
memory usage: 4.8+ MB


In [11]:
# filter genres
genres_to_keep = ["Hiphop", "trap", "techno", "RnB", "Rap", "Pop" , "Emo"]
idx = data['genre'].isin(genres_to_keep)
data = data.loc[idx]

In [12]:
# there are duplicates in the data - some songs fall under multiple genres 
data[id_col].value_counts()

0x66nbDEEZtWx4qJdILucR    6
75vNXX9hCL0yCAbYWKBBsB    6
4MLS3xVoEtru6VQM8Fp2Z4    6
25GcNk1AqlRmtcdWONCc3z    5
19I2mi8kcf0fmoDphv1SOC    5
                         ..
65NwOZqoXny4JxqAPlfxRF    1
4bdJHQp90ymjEIkkYhcENB    1
2yUbCEiaolfSMluDo9RMmG    1
1QRJb5VY3sIkCKcTuIKCrk    1
3lKz1NkKMSrDPDSfwDB2v4    1
Name: id, Length: 13566, dtype: int64

In [13]:
# drop duplicates. we will keep the first one
not_target_cols = [c for c in data.columns if c != target_col]

data = data.drop_duplicates(subset = [id_col], keep='first')
data.shape

(13566, 15)

In [14]:
# verify ids are now unique
data[id_col].nunique()

13566

# Shuffle Data

In [15]:
# shuffle data
data = data.sample(frac=1, random_state=42)
data.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,genre
13386,0.389,0.665,0,-6.169,1,0.0644,0.1780,0.000732,0.1160,0.199,117.055,7lGKEWMXVWWTt3X71Bv44I,193547,4,Emo
15730,0.796,0.672,4,-6.255,0,0.2400,0.0885,0.000403,0.0693,0.218,154.961,1XVOUl9oLYeltxB0NXarcy,153365,4,Rap
18527,0.767,0.864,10,-1.847,0,0.1330,0.0116,0.000000,0.0957,0.747,170.950,0nKZeSrqH9u31NLoTUsYBR,257907,4,Hiphop
14106,0.899,0.586,11,-7.866,0,0.3430,0.2790,0.000000,0.0836,0.492,150.002,3qN5qMTKyEEmiTZD38BNTT,214467,4,Rap
14071,0.741,0.370,8,-8.437,0,0.3240,0.2400,0.000000,0.1180,0.130,116.060,4lFfMRH0YH4pW5gczTDbNC,219136,4,Emo


# Save Main Data File

In [16]:
data.to_csv(outp_fname, index=False)

# Train Test Split

In [17]:
from sklearn.model_selection import train_test_split
test_size = 0.1

data_train, data_test = train_test_split(data, test_size=test_size, random_state=42)
print(data_train.shape, data_test.shape)

data_train.to_csv(outp_train_fname, index=False)
data_test.drop(columns=[target_col]).to_csv(outp_test_fname, index=False)
data_test[[id_col, target_col]].to_csv(outp_test_key_fname, index=False)

(12209, 15) (1357, 15)


# JSON inference request instance

In [18]:
instance = data_test.replace({np.nan:None}).drop(columns=[target_col]).reset_index(drop=True).loc[0].to_dict()
infer_req_instance_dict = {  "instances": [ {**instance}, ] }
pprint.pprint(infer_req_instance_dict)
   
with open(outp_infer_instances, 'w', encoding='utf8') as f:
    json.dump(infer_req_instance_dict, f, indent=2, ensure_ascii=False)

{'instances': [{'acousticness': 0.0599,
                'danceability': 0.797,
                'duration_ms': 224240,
                'energy': 0.469,
                'id': '5IZc3KIVFhjzJ0L2kiXzUl',
                'instrumentalness': 0.0,
                'key': 8,
                'liveness': 0.0753,
                'loudness': -8.112,
                'mode': 1,
                'speechiness': 0.0646,
                'tempo': 206.114,
                'time_signature': 4,
                'valence': 0.481}]}
